## <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea">Assignment 6: AllLife Bank Customer Segmentation</span>

&nbsp;

<p align="center" style="font-family: Arial;color:#0e92ea;font-size:1em;">
AllLife Bank wants to focus on its credit card customer base in the next financial year. They have been advised by their marketing research team, that the penetration in the market can be improved. Based on this input, the Marketing team proposes to run personalized campaigns to target new customers as well as upsell to existing customers. Another insight from the market research was that the customers perceive the support services of the back poorly. Based on this, the Operations team wants to upgrade the service delivery model, to ensure that customer queries are resolved faster. Head of Marketing and Head of Delivery both decide to reach out to the Data Science team for help
</p>

##  <span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#0e92ea"> Objective:</span>
<p align="center" style="font-family: Arial;color:#0e92ea;font-size:1em;">
To identify different segments in the existing customer, based on their spending patterns as well as past interaction with the bank, using clustering algorithms, and provide recommendations to the bank on how to better market to and service these customers.
</p>

##  <span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#0e92ea">Data Dictionary:</span>

<ol style="font-family: Arial;color:#0e92ea;font-size:1em;">
    <li>Sl_No: Primary key of the records</li>
    <li>Customer Key: Customer identification number</li>
    <li>Average Credit Limit: Average credit limit of each customer for all credit cards</li>
    <li>Total credit cards: Total number of credit cards possessed by the customer</li>
    <li>Total visits bank: Total number of visits that customer made (yearly) personally to the bank</li>
    <li>Total visits online: Total number of visits or online logins made by the customer (yearly)</li>
    <li>Total calls made: Total number of calls made by the customer to the bank or its customer service department (yearly)</li>
</ol>

##  <span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#0e92ea"> Contents:</span>

<ol style="font-family: Arial;color:#0e92ea;font-size:1em;">
    <li>Exploratory Data Analysis and Insights</li>
    <li>Data pre-processing</li>
    <li>Model building</li>
    <li>Model building - Oversampled data</li>
    <li>Model building - Undersampled data</li>
    <li>Hyperparameter tuning using random search</li>
    <li>Model Performances</li>
    <li>Productionize the model</li>
    <li>Recommendations</li>
</ol>

In [3]:
import os
import warnings
import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

from matplotlib import gridspec
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
#from xgboost import XGBClassifier


from sklearn import tree

from os import system
from IPython.display import Image


warnings.filterwarnings('ignore')                                       # Ignore/Do not display warnings

main_data = pd.read_excel('Credit+Card+Customer+Data.xlsx')

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 1 and 2: Exploratory Data Analysis and Insights and Data Processing

In [5]:
main_data.head(10)

,Sl_No,Customer Key,Avg_Credit_Limit,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
0,1,87073,100000,2,1,1,0
1,2,38414,50000,3,0,10,9
2,3,17341,50000,7,1,3,4
3,4,40496,30000,5,1,1,4
4,5,47437,100000,6,0,12,3
5,6,58634,20000,3,0,1,8
6,7,48370,100000,5,0,11,2
7,8,37376,15000,3,0,1,1
8,9,82490,5000,2,0,2,2
9,10,44770,3000,4,0,1,7


In [12]:
'''
Input:
Pandas DataFrame

Output:
Displays DataFrame structure 
(columns, nulls and non nulls counts and percentage highlighing columns with most number of nulls)

Retunrs:
N/A
'''
def info(dataFrame):
    print(f"{dataFrame.shape[0]} Rows x {dataFrame.shape[1]} Columns")
    nulls_series      = dataFrame.isna().sum()         # Get a series counting number of empty values for each column
    nonnulls_series   = dataFrame.notnull().sum()      # Get a series counting number of non empty valuesfor each column 
    nulls_percentage  = ((nulls_series * 100)/(nulls_series + nonnulls_series)).astype(float)
    column_datatypes  = dataFrame.dtypes               # Get a series containing data types for each column 
    
    series_arr    = [nulls_series, nonnulls_series, nulls_percentage, column_datatypes]
    col_names_arr = ["Missing", "Non-Nulls","Missing %", "Type"]
    
    nulls_count_df = pd.concat(
        objs = series_arr,
        axis = 1,
        keys = col_names_arr, 
        sort = True)
    
    cm = sns.light_palette("red", as_cmap=True)
    display(nulls_count_df.style.background_gradient(cmap=cm, subset=pd.IndexSlice[:, col_names_arr[1]]).format(formatter={(col_names_arr[1]): "{:.2f}%"})) 

In [13]:
info(main_data)

660 Rows x 7 Columns


,Missing,Non-Nulls,Missing %,Type
Avg_Credit_Limit,0,660.00%,0.000000,int64
Customer Key,0,660.00%,0.000000,int64
Sl_No,0,660.00%,0.000000,int64
Total_Credit_Cards,0,660.00%,0.000000,int64
Total_calls_made,0,660.00%,0.000000,int64
Total_visits_bank,0,660.00%,0.000000,int64
Total_visits_online,0,660.00%,0.000000,int64
